In [1]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 
import os
import fnmatch

In [2]:
train_path = "Demo_dataset"
train_chinese = os.listdir(train_path)
df_train = pd.DataFrame(columns=["character", "filename", "label"])
len(train_chinese)

8

In [3]:
k = 0
for i in train_chinese:
    if i != ".DS_Store":
        l = os.listdir(train_path + '/' + i)
        l = fnmatch.filter(l, '*.png')
        if len(l) > 200:
            l = l[:200]
        label = [k] * len(l)
        temp = pd.DataFrame({
            "character": i,
            "filename": l,
            "label": label
        })
        #print(temp)
        df_train = pd.concat([df_train, temp])      
    k += 1
        

In [4]:
df_train

,character,filename,label
0,理,176.png,0
1,理,88.png,0
2,理,162.png,0
3,理,189.png,0
4,理,77.png,0
...,...,...,...
195,鸫,93.png,7
196,鸫,78.png,7
197,鸫,186.png,7
198,鸫,192.png,7


In [5]:
train_array = np.zeros((df_train.shape[0], 128, 128, 3))
for i in range(df_train.shape[0]):
    if df_train.iloc[i]['filename'] != ".DS_Store":
        image_path = train_path + '/' + df_train.iloc[i]['character'] + '/' + df_train.iloc[i]['filename']
        # get grayscale image
        # print(image_path)
        image = cv2.imread(image_path)
        # remove noise
        out = cv2.medianBlur(image, 5)
        # normalize the image
        resultimage = np.zeros(out.shape)
        normalizedimage = cv2.normalize(out, resultimage, 0, 255, cv2.NORM_MINMAX)
        # resize the image 
        resizeimage = cv2.resize(normalizedimage, (128,128))

        blue, green, red = cv2.split(resizeimage) 
        # apply principal component analysis
        pca = PCA(50)
        red_transformed = pca.fit_transform(red)
        red_inverted = pca.inverse_transform(red_transformed)

        #Applying to Green channel and then applying inverse transform to transformed array.
        green_transformed = pca.fit_transform(green)
        green_inverted = pca.inverse_transform(green_transformed)

        #Applying to Blue channel and then applying inverse transform to transformed array.
        blue_transformed = pca.fit_transform(blue)
        blue_inverted = pca.inverse_transform(blue_transformed)

        img_compressed = (np.dstack((red_inverted, green_inverted, blue_inverted))).astype(np.uint8)
        train_array[i] = img_compressed



In [6]:
train_array.shape

(1600, 128, 128, 3)

In [7]:
X = np.array(train_array).reshape(len(train_array),-1)
y = np.asarray(df_train["label"].astype('int'))

In [8]:
# scaling the data
X = X / 255.0

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [10]:
from sklearn.svm import SVC
svc = SVC(kernel='linear', C = 0.1)
svc.fit(X_train, y_train)

SVC(C=0.1, kernel='linear')

In [11]:
y_predicted = svc.predict(X_test)

In [12]:
from sklearn.metrics import accuracy_score
print("Accuracy on unknown data is", accuracy_score(y_test, y_predicted))

Accuracy on unknown data is 0.8725
